In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import Python_Chapter_14
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")

# Data
We import bitcoin data from Yahoo finance from 2021.

In [2]:
btc = yf.Ticker('BTC-USD').history(start='2021-01-01', end='2021-12-01')['Close']

In [3]:
btc_df = pd.DataFrame(btc)
btc_df['Return'] = btc.pct_change()
btc_df['Target'] = -btc.diff(-1).apply(np.sign).replace(0, -1)
btc_df = btc_df.dropna()
btc = btc_df['Close']
target_positions = btc_df['Target']
returns = btc_df['Return']

# Bet timing
We derive the bet timing when positions flatten or flip.

In [4]:
Python_Chapter_14.bet_timing(target_positions)

DatetimeIndex(['2021-01-03', '2021-01-04', '2021-01-08', '2021-01-12',
               '2021-01-14', '2021-01-17', '2021-01-18', '2021-01-21',
               '2021-01-22', '2021-01-23',
               ...
               '2021-11-16', '2021-11-17', '2021-11-18', '2021-11-20',
               '2021-11-22', '2021-11-23', '2021-11-24', '2021-11-25',
               '2021-11-26', '2021-11-29'],
              dtype='datetime64[ns]', name='Date', length=185, freq=None)

# Holding Period 
Derives avgerage holding period (in days) using avg entry time pairing algo

In [5]:
holding, mean = Python_Chapter_14.holding_period(target_positions)
holding

,dT,w
2021-01-03,2.0,1.0
2021-01-04,1.0,1.0
2021-01-08,4.0,1.0
2021-01-12,4.0,1.0
2021-01-14,2.0,1.0
...,...,...
2021-11-23,1.0,1.0
2021-11-24,1.0,1.0
2021-11-25,1.0,1.0
2021-11-26,1.0,1.0


In [6]:
mean

1.7945945945945947

#  HHI concentration
Derives the algorithm for deriving hhi concentration

In [7]:
positive_returns_concentration, negative_returns_concentration, monthly_returns_concentration = Python_Chapter_14.hhi_concentration(returns)

In [8]:
positive_returns_concentration

0.004531515724669987

In [9]:
negative_returns_concentration

0.005214430443380818

In [10]:
monthly_returns_concentration

0.00010100190280370181

# Compute Draw Downs And Time Under Water
Computes series of drawdowns and the time under water associated with them

In [11]:
drawdowns, time_under_water = Python_Chapter_14.compute_drawdowns_time_under_water(btc, True)
drawdowns

Date
2021-01-03      810.109375
2021-01-08    10365.062500
2021-02-09     1562.921875
2021-02-11      803.816406
2021-02-14      772.230469
2021-02-17      469.210938
2021-02-21    12402.175781
2021-03-11      473.031250
2021-03-13     9538.925781
2021-04-13    33696.109375
2021-10-20     7510.449219
2021-11-08    13997.062500
dtype: float64

In [12]:
time_under_water

Date
2021-01-03    0.013690
2021-01-08    0.087613
2021-02-09    0.005476
2021-02-11    0.008214
2021-02-14    0.008214
2021-02-17    0.010952
2021-02-21    0.049282
2021-03-11    0.005476
2021-03-13    0.084875
2021-04-13    0.520202
2021-10-20    0.052020
dtype: float64

In [34]:
def compute_drawdowns_time_under_water(
        series: pd.Series, # series of returns or dollar performance
        dollars=False
    ): # returns or dollar performance

    series_df = series.to_frame('PnL').reset_index(names='Datetime') # convert to DataFrame
    series_df['HWM'] = series.expanding().max().values # find max of expanding window

    def process_groups(group): # proces drawdowns

        if len(group) <= 1: # check if there is a drawdown 
            return None
        
        result = pd.Series()
        result.loc['Start'] = group['Datetime'].iloc[0] # find drawdown beginning
        result.loc['Stop'] = group['Datetime'].iloc[-1] # find drawdown ending
        result.loc['HWM'] = group['HWM'].iloc[0] # find drawdown high watermark
        result.loc['Min'] = group['PnL'].min() # find the maximum drawdown 
        result.loc['Min. Time'] = group['Datetime'][group['PnL'] == group['PnL'].min()].iloc[0] # find the maximum drawdown time

        return result
    
    groups = series_df.groupby('HWM') # group by high water mark
    drawdown_analysis = pd.DataFrame() # initiate dataframe    

    for _, group in groups:
        drawdown_analysis = drawdown_analysis.append(process_groups(group), ignore_index=True) # process and aggregate drawdowns

    if dollars:
        drawdown = drawdown_analysis['HWM'] - drawdown_analysis['Min'] # calculate drawdowns
    else:
        drawdown = 1 - drawdown_analysis['Min'] / drawdown_analysis['HWM'] # calculate drawdowns

    drawdown.index = drawdown_analysis['Start'] # set index
    drawdown.index.name = 'Datetime' # set index name

    time_under_water = ((drawdown_analysis['Stop'] - drawdown_analysis['Start']) / np.timedelta64(1, 'Y')).values # convert time under water to years
    time_under_water = pd.Series(time_under_water, index=drawdown_analysis['Start']) # create Series

    return drawdown, time_under_water, drawdown_analysis

drawdowns, time_under_water, drawdown_analysis = compute_drawdowns_time_under_water(btc, True)

In [39]:
fig = px.line(btc)
fig.update_layout({
    'plot_bgcolor': 'rgba(0, 0, 0, 0)',
    'paper_bgcolor': 'rgba(0, 0, 0, 0)'
    },
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False)
)
for i in range(len(drawdown_analysis)):
    current_drawdown = drawdown_analysis.iloc[i]

    fig.add_shape(
        type='line',
        x0=current_drawdown['Start'],
        y0=btc[current_drawdown['Start']],
        x1=current_drawdown['Stop'],
        y1=btc[current_drawdown['Start']],
        line=dict(color='Red', dash='dash')
    )
    fig.add_shape(
        type='line',
        x0=current_drawdown['Min. Time'],
        y0=btc[current_drawdown['Min. Time']],
        x1=current_drawdown['Min. Time'],
        y1=current_drawdown['HWM'],
        line=dict(color='Green', dash='dash')              
    )

fig.show()

In [40]:
import scipy.stats as ss

"""
    Calculates the probabilistic Sharpe ratio (PSR) that provides an adjusted estimate of SR,
    by removing the inflationary effect caused by short series with skewed and/or
    fat-tailed returns.

    Given a user-defined benchmark Sharpe ratio and an observed Sharpe ratio,
    PSR estimates the probability that SR ̂is greater than a hypothetical SR.
    - It should exceed 0.95, for the standard significance level of 5%.
    - It can be computed on absolute or relative returns.

    :param observed_sr: (float) Sharpe ratio that is observed.
    :param benchmark_sr: (float) Sharpe ratio to which observed_SR is tested against.
    :param number_of_returns: (int) Times returns are recorded for observed_SR.
    :param skewness_of_returns: (float) Skewness of returns (0 by default).
    :param kurtosis_of_returns: (float) Kurtosis of returns (3 by default).
    :return: (float) Probabilistic Sharpe ratio.
"""
def probabilistic_sharpe_ratio(observed_sr: float, benchmark_sr: float, number_of_returns: int,
                               skewness_of_returns: float = 0, kurtosis_of_returns: float = 3) -> float:

    test_value = ((observed_sr - benchmark_sr) * np.sqrt(number_of_returns - 1)) / \
                  ((1 - skewness_of_returns * observed_sr + (kurtosis_of_returns - 1) / \
                    4 * observed_sr ** 2)**(1 / 2))

    if np.isnan(test_value):
        warnings.warn('Test value is nan. Please check the input values.', UserWarning)
        return test_value

    if isinstance(test_value, complex):
        warnings.warn('Output is a complex number. You may want to check the input skewness (too high), '
                      'kurtosis (too low), or observed_sr values.', UserWarning)

    if np.isinf(test_value):
        warnings.warn('Test value is infinite. You may want to check the input skewness, '
                      'kurtosis, or observed_sr values.', UserWarning)

    probab_sr = ss.norm.cdf(test_value)

    return probab_sr

In [76]:
import plotly.graph_objects as go

def f_psr(x, y):
    return probabilistic_sharpe_ratio(1.5, 1.0, y, x, 3.0)

x = np.linspace(-1.5, 1, 100)
y = np.linspace(1, 30, 100)
z = np.zeros((len(x), len(y)))
for i in range(len(x)):
    for j in range(len(y)):
        z[i, j] = f_psr(x[i], y[j])

fig = go.Figure(data=[go.Surface(z=z, x=y, y=x,
                                 contours = {
        "z": {"show": True, "start": z.min(), "end": z.max(), "size": 0.08}
    },)])
fig.update_layout(
    {
                   'plot_bgcolor': 'rgba(0, 0, 0, 0)',
                   'paper_bgcolor': 'rgba(0, 0, 0, 0)',
                  }, scene=dict(
                                xaxis_title='Sample Length',
                                yaxis_title='Skewness',
                                zaxis_title='Probabilistic Sharpe Ratio',
                                xaxis = dict(
                                showbackground=False),
                                yaxis = dict(
                                showbackground=False),
                                zaxis = dict(
                                showbackground=False)
                               ))

fig.show()

In [77]:
def f_bsr(x, y):
    return x * ((1 - np.euler_gamma) * ss.norm.ppf(1 - 1 / y) + \
            np.euler_gamma * ss.norm.ppf(1 - 1 / y * np.e ** (-1)))

x = np.linspace(0.1, 1, 100)  
y = np.linspace(1, 30, 100)     
z = np.zeros((len(x), len(y)))
for i in range(len(x)):
    for j in range(len(y)):
        z[i, j] = f_bsr(x[i], y[j])

fig = go.Figure(data=[go.Surface(z=z, x=y, y=x,
                                 contours = {
        "z": {"show": True, "start": z.min(), "end": z.max(), "size": 0.1}
    },)])
fig.update_layout({
                   'plot_bgcolor': 'rgba(0, 0, 0, 0)',
                   'paper_bgcolor': 'rgba(0, 0, 0, 0)',
                  }, scene=dict(
                                xaxis_title='Number Of Trials',
                                yaxis_title='Trials\' Variance',
                                zaxis_title='Benchmark Sharpe Ratio',
                                xaxis = dict(
                                showbackground=False),
                                yaxis = dict(
                                showbackground=False),
                                zaxis = dict(
                                showbackground=False)
                               ))
fig.show()